In [ ]:
import numpy as np
import pandas as pd
import pymc3 as pm
from scipy.stats import norm

In [ ]:
df = pd.read_csv("/content/part1.csv")

## 데이터 소개

* 한 기업에서, 국제 시장 진출을 위해 10개 나라(`A, B, ..., J`)를 대상으로 100일 간 제품 시범 운영
* 각 나라마다 진출 후보 도시의 숫자가 다름


In [ ]:
# 총 도시의 숫자
n_cities, = df.city_id.unique().shape
n_cities

143

In [ ]:
# 각 국가별 도시의 숫자
(
  df[['country_id', 'city_id']]
  .drop_duplicates()
  .groupby('country_id')
  .count()
)

* 데이터는, 각 [국가, 도시] 마다 100일 간 관찰한 순수익/이윤(`profit`)

In [ ]:
df.dtypes

country_id     object
city_id        object
profit        float64
dtype: object

## 1. 각 국가별 평균 수익률을 계산하고 95% 신뢰구간을 계산

* Assume normally distributed profit with mean $p_i$ for each country $i$ with variation $s_i^2$.
* Using bootstrap (but could use one of many other analytical solutions)

In [ ]:
def bootstrap(df: pd.DataFrame, B=1000, confidence=0.95) -> pd.DataFrame:
    """Bootstrap profit"""
    profit = df.profit
    sample = np.random.choice(profit, size=(int(B), len(profit))).mean(axis=1)

    est = profit.mean()
    se = sample.std()

    alpha = (1 - confidence) / 2
    z = norm.ppf([alpha, 1 - alpha])

    lb, ub = est + z * se

    return pd.DataFrame([{
        'estimate': est,
        'std_err': se,
        'bsci_lb': lb,
        'bsci_ub': ub,
    }])

In [ ]:
(
    df
    .groupby(['country_id'])
    .apply(bootstrap)
)

,,estimate,std_err,bsci_lb,bsci_ub
country_id,,,,,
A,0,9896.792195,78.894252,9742.162302,10051.422088
B,0,827.489033,112.869160,606.269545,1048.708521
C,0,2564.343213,112.419002,2344.006017,2784.680408
D,0,4719.311179,89.366297,4544.156455,4894.465902
E,0,4408.554384,85.211800,4241.542326,4575.566442
F,0,6144.786339,90.060574,5968.270856,6321.301821
G,0,3970.277311,124.581136,3726.102771,4214.451851
H,0,5738.696122,142.821475,5458.771175,6018.621069
I,0,6773.143955,77.269680,6621.698166,6924.589744


## 2. 결과를 본 임원들 왈: "평균이 제일 높은 나라 하나에 100% 진출하자!". 

* 사내 유일한 데이터 사이언티스트로서 당신의 반응/제안은?

### 답 가이드라인
* 각 국가마다 포함한 도시의 숫자가 다름. 평균을 일반화 할 수는 없음.
* 도시 숫자가 다르기 때문에 추정치의 정확도도 각각 다름
* 데이터의 불확실성을 고려했을 때, 10개의 국가 중 "평균이 가장 높게 나온 나라"를 선택한다는 것은 오히려 운 좋게 상향 편향 (upward bias) 된 국가를 선택하는 잘못이 될 수 있음
* 특히 두 개 국가(`A`와 `J`)가 매우 가까운 평균 이윤을 달성했다는 점에서, `J` (\$9,824.24)말고 `A`(\$9896.79)에 진출 하는 것의 타당성 우려 

## 3. 임원들 왈: "그러면 통계적으로 유의미하게 이윤이 $10,000 이상인 도시에만 진출하자!"

* 신뢰수준 95%에서 추정 이윤이 $10,000 이상인 **도시** 찾기

### 답 가이드라인:

* 도시 찾기:

In [ ]:
threshold = 10000
(
    df
    .groupby(['country_id', 'city_id'])
    .apply(bootstrap)
    .assign(significant=lambda d: d.bsci_lb >= threshold)
    .loc[lambda d: d.significant]
    .sort_values('estimate', ascending=False)
)

estimate     std_err  ...       bsci_ub  significant
country_id city_id                            ...                           
A          a02     0  16889.5987  241.121971  ...  17362.189080         True
           a05     0  14534.7150  240.740103  ...  15006.556932         True
J          j10     0  14303.1586  242.635902  ...  14778.716228         True
F          f05     0  13755.5318  245.485353  ...  14236.674250         True
J          j08     0  13241.0068  214.493967  ...  13661.407250         True
           j14     0  12882.4972  252.952875  ...  13378.275726         True
           j09     0  12572.9120  218.066289  ...  13000.314073         True
A          a08     0  12286.2988  256.365678  ...  12788.766296         True
J          j12     0  11766.8433  269.577278  ...  12295.205056         True
F          f02     0  11538.2496  258.174774  ...  12044.262860         True
A          a17     0  11447.9946  259.717848  ...  11957.032229         True
J          j03     0  11127.8873  260.198079  ...  11637.866164         True
A          a13     0  10990.3122  257.768569  ...  11495.529311         True
H          h07     0  10714.7196  234.635034  ...  11174.595817         True
I          i06     0  10604.4801  256.168225  ...  11106.560595         True
J          j05     0  10602.2638  244.092841  ...  11080.676978         True
           j16     0  10525.7585  252.681592  ...  11021.005320         True

[17 rows x 5 columns]

## 4. 이전 결과대로 시장 진출 할 경우 우려는?

* 같은 국가 내에 있는 도시는 독립적이지 않을 가능성이 큼
* 총 143개의 도시가 각각 독립이라 가정하더라도, 95% 신뢰수준의 가설검증을 143번 했을 때, 실제 이윤이 \$10,000 이상이 되지 **않지만** 통계적으로 유의미한 결과는 약 $143 \times 0.05 \approx 7$개까지도 기대할 수 있음.  
* 위 결과 17개의 도시가 올라온 것을 봤을 때, 이 중 거의 절반은 실제 이윤이 \$10,000이 넘지 않을수도 있음.

In [ ]:
143 * 0.05

7.15

### [고급] 5. 다음의 베이지안 확률모형으로 각 도시의 이윤에 대한 사후 확률 분포를 계산해 보기

* 현재 강의 내용만으로는 진행이 어려울 수 있음
* 비슷한 다른 예제를 참고하며 배울 수 있는 기회: 
    * https://docs.pymc.io/en/v3/nb_examples/index.html
    * https://nbviewer.org/github/fonnesbeck/multilevel_modeling/blob/master/multilevel_modeling.ipynb?create=1